In [1]:
!pip install pymysql

In [2]:
import pymysql
import pandas as pd
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [4]:
import sqlite3

conn = sqlite3.connect('example_2.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    reliability FLOAT
)
''')

cursor.execute('''
CREATE TABLE datasets (
    id INTEGER PRIMARY KEY AUTOINCREMENT
)
''')

cursor.execute('''
CREATE TABLE categories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name TEXT NOT NULL,
    dataset_id INTEGER NOT NULL,
    FOREIGN KEY (dataset_id) REFERENCES datasets(id)
)
''')

cursor.execute('''
CREATE TABLE documents (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    document_text TEXT NOT NULL,
    dataset_id INTEGER NOT NULL,
    predicted_category_id INTEGER,
    FOREIGN KEY (dataset_id) REFERENCES datasets(id),
    FOREIGN KEY (predicted_category_id) REFERENCES categories(id)
)
''')

cursor.execute('''
CREATE TABLE annotatorLabels (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    document_id INTEGER NOT NULL,
    category_id INTEGER NOT NULL,
    annotator_user_id INTEGER NOT NULL,
    FOREIGN KEY (document_id) REFERENCES documents(id),
    FOREIGN KEY (category_id) REFERENCES categories(id),
    FOREIGN KEY (annotator_user_id) REFERENCES users(id)
)
''')

cursor.executemany('''
INSERT INTO users (id, reliability) VALUES (?, ?)
''', [
    (1, 0.95),
    (2, 0.80),
    (3, 0.75),
    (4, 0.90),
    (5, 0.85)
])

cursor.executemany('''
INSERT INTO datasets (id) VALUES (?)
''', [
    (1,),
    (2,),
    (3,)
])

cursor.executemany('''
INSERT INTO categories (id, category_name, dataset_id) VALUES (?, ?, ?)
''', [
    (1, 'Sports', 1),
    (2, 'Politics', 1),
    (3, 'Technology', 2),
    (4, 'Health', 2),
    (5, 'Entertainment', 3)
])

cursor.executemany('''
INSERT INTO documents (id, document_text, dataset_id, predicted_category_id) VALUES (?, ?, ?, ?)
''', [
    (1, 'The football match was thrilling with a last-minute goal.', 1, 1),
    (2, 'The new policy has caused quite a stir among the citizens.', 1, 2),
    (3, 'Innovations in AI are driving the tech industry forward.', 2, 3),
    (4, 'A new vaccine has been developed to combat the virus.', 2, 4),
    (5, 'The latest movie release has broken box office records.', 3, 5)
])

cursor.executemany('''
INSERT INTO annotatorLabels (id, document_id, category_id, annotator_user_id) VALUES (?, ?, ?, ?)
''', [
    (1, 1, 1, 1),
    (2, 2, 2, 2),
    (3, 3, 3, 3),
    (4, 4, 4, 4),
    (5, 5, 5, 5)
])

conn.commit()
conn.close()


In [5]:
def get_db_connection():
    conn = sqlite3.connect('example_2.db')
    return conn, conn.cursor()

def fetch_data():
    conn, cursor = get_db_connection()
    query = '''
        SELECT d.id as document_id, d.document_text, al.category_id, u.reliability
        FROM documents d
        JOIN annotatorLabels al ON d.id = al.document_id
        JOIN users u ON al.annotator_user_id = u.id
    '''
    cursor.execute(query)
    data = cursor.fetchall()
    conn.close()
    return data

def compute_weighted_labels(df):
    weighted_labels = df.groupby('document_id').apply(
        lambda x: x.groupby('category_id')['reliability'].sum().idxmax()
    ).reset_index(name='weighted_category_id')
    return weighted_labels



In [6]:
data = fetch_data()

columns = ['document_id', 'document_text', 'category_id', 'reliability']
df = pd.DataFrame(data, columns=columns)

weighted_labels = compute_weighted_labels(df)

df = df.drop_duplicates(subset='document_id')

df = df.merge(weighted_labels, on='document_id')

documents = df['document_text'].tolist()
labels = df['weighted_category_id'].tolist()

In [7]:
tokenized_documents = [doc.split() for doc in documents]

word2vec_model = Word2Vec(sentences=tokenized_documents, vector_size=100, window=5, min_count=1, workers=4)

def document_vector(doc):
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) == 0:
        return np.zeros(100)
    return np.mean(word2vec_model.wv[doc], axis=0)

X = np.array([document_vector(doc) for doc in tokenized_documents if len(doc) > 0])
y = LabelEncoder().fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

predicted_categories = classifier.predict(X)

print(predicted_categories)


[0 0 2 3 0]


In [8]:
def update_predicted_categories(document_ids, predicted_categories):
    conn, cursor = get_db_connection()
    for doc_id, category in zip(document_ids, predicted_categories):
        cursor.execute('UPDATE documents SET predicted_category_id = ? WHERE id = ?', (int(category), int(doc_id)))
    conn.commit()
    conn.close()

update_predicted_categories(df['document_id'].tolist(), predicted_categories)

print("Database updated with predicted categories.")

Database updated with predicted categories.


In [9]:
def view_table(table_name):
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    rows = cursor.fetchall()
    conn.close()
    return rows



In [10]:
documents = view_table('documents')
print("Documents Table:")
for row in documents:
    print(row)


categories = view_table('categories')
print("\nCategories Table:")
for row in categories:
    print(row)

Documents Table:
(1, 'The football match was thrilling with a last-minute goal.', 1, 0)
(2, 'The new policy has caused quite a stir among the citizens.', 1, 3)
(3, 'Innovations in AI are driving the tech industry forward.', 2, 2)
(4, 'A new vaccine has been developed to combat the virus.', 2, 3)
(5, 'The latest movie release has broken box office records.', 3, 0)

Categories Table:
(1, 'Sports', 1)
(2, 'Politics', 1)
(3, 'Technology', 2)
(4, 'Health', 2)
(5, 'Entertainment', 3)
